In [1]:
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dropout, Dense
import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math
import time
import itertools

Using TensorFlow backend.


In [2]:
#Read first dataset
dataset1 = pd.read_csv('./mushrooms.csv')
#np.count_nonzero(dataset1['Outcome'])/len(dataset1['Outcome'])

In [3]:
continuous=['class']
dataset1_dummy = pd.DataFrame()
for col in dataset1.columns:
    if col not in continuous:
        uniques = np.unique(dataset1[col])
        for unique in uniques:
            name = str(col)+'_'+str(unique)
            dataset1_dummy[name]=(dataset1[col]==unique)
    elif col=='class':
        dataset1_dummy['class1']=(dataset1[col]=='p')
    else:
        dataset1_dummy[col]=dataset1[col]
(dataset1_dummy)

,class1,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,True,False,False,False,False,False,True,False,False,True,...,True,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,True,False,False,True,...,False,False,False,False,True,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
3,True,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,True,False,False,True,...,False,False,False,False,True,False,False,False,False,False
5,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
6,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
7,False,True,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,False
8,True,False,False,False,False,False,True,False,False,False,...,False,True,False,False,True,False,False,False,False,False
9,False,True,False,False,False,False,False,False,False,True,...,True,False,False,False,False,False,True,False,False,False


In [4]:
'''print(len(dataset1_dummy))
indexes = dataset1_dummy[dataset1_dummy['class1']==1].index
idp = dataset1_dummy[dataset1_dummy['class1']==0].iloc[0:len(indexes)].index
indexes = indexes.append(idp)
dataset1 = dataset1_dummy.iloc[indexes]
dataset1 = dataset1_dummy.sort_index()
np.count_nonzero(dataset1_dummy['class1'])/len(dataset1_dummy['class1'])'''

"print(len(dataset1_dummy))\nindexes = dataset1_dummy[dataset1_dummy['class1']==1].index\nidp = dataset1_dummy[dataset1_dummy['class1']==0].iloc[0:len(indexes)].index\nindexes = indexes.append(idp)\ndataset1 = dataset1_dummy.iloc[indexes]\ndataset1 = dataset1_dummy.sort_index()\nnp.count_nonzero(dataset1_dummy['class1'])/len(dataset1_dummy['class1'])"

In [5]:
#Read Second dataset
dataset2 = pd.read_csv('adult.csv')
dataset2.head(5)

,age,workclass,fnlwgt,education,education_num,marital_state,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_county,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
continuous = ['age','fnlwgt','capital_gain','capital_loss','hours_per_week','class']
dataset2

,age,workclass,fnlwgt,education,education_num,marital_state,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_county,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [7]:
dataset2_dummy = pd.DataFrame()
for col in dataset2.columns:
    if col not in continuous:
        uniques = np.unique(dataset2[col])
        for unique in uniques:
            name = str(col)+'_'+str(unique)
            dataset2_dummy[name]=(dataset2[col]==unique)
    elif col=='class':
        dataset2_dummy['class2']=(dataset2[col]==' <=50K')
    else:
        dataset2_dummy[col]=dataset2[col]
(dataset2_dummy)

,age,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,...,native_county_ Puerto-Rico,native_county_ Scotland,native_county_ South,native_county_ Taiwan,native_county_ Thailand,native_county_ Trinadad&Tobago,native_county_ United-States,native_county_ Vietnam,native_county_ Yugoslavia,class2
0,39,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
1,50,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
2,38,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
3,53,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,28,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5,37,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
6,49,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
7,52,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
8,31,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
9,42,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [8]:
indexes = dataset2_dummy[(dataset2_dummy['class2'])==0].index
idp = dataset2_dummy[(dataset2_dummy['class2'])==1].iloc[0:len(indexes)].index
indexes=indexes.append(idp)
dataset2_dummy = dataset2_dummy.iloc[indexes]
dataset2_dummy


,age,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,...,native_county_ Puerto-Rico,native_county_ Scotland,native_county_ South,native_county_ Taiwan,native_county_ Thailand,native_county_ Trinadad&Tobago,native_county_ United-States,native_county_ Vietnam,native_county_ Yugoslavia,class2
7,52,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
8,31,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
9,42,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
10,37,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
11,30,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
14,40,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19,43,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
20,40,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
25,56,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
27,54,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [9]:
dataset1_back = dataset1
dataset1 = dataset1_dummy.sort_index()
dataset2_back = dataset2
dataset2 = dataset2_dummy.sort_index()
#np.count_nonzero(dataset2['class'])/len(dataset2['class'])

In [10]:
size = len(dataset1)+len(dataset2)
dataset_new = pd.DataFrame(columns=list(dataset1.columns)+list(dataset2.columns)+list(['classf']),index=range(size))
dataset_new

,class1,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,...,native_county_ Scotland,native_county_ South,native_county_ Taiwan,native_county_ Thailand,native_county_ Trinadad&Tobago,native_county_ United-States,native_county_ Vietnam,native_county_ Yugoslavia,class2,classf
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for column in dataset2.columns:
    dataset_new[column][0:len(dataset2)] = dataset2[column]
    if column=='class2':
        dataset_new['classf'][0:len(dataset2)] = dataset2[column]+0

for column in dataset1.columns:
    dataset_new[column][len(dataset2):size] = dataset1[column]
    if column=='class1':
        dataset_new['classf'][len(dataset2):size] = dataset1[column]+2
        



'''dataset_new = pd.DataFrame(columns=list(dataset1.columns)+list(dataset2.columns))
for column in dataset2.columns:
    dataset_new[column] = dataset2[column]
dataset_new.index = range(len(dataset_new))
indnan = 0
while indnan in dataset_new.index:
    max = indnan+len(dataset1)
    if max in dataset_new.index: 
        #print(indnan)
        for column in dataset1.columns:
            dataset_new[column][indnan:indnan+len(dataset1)] = dataset1[column]
    indnan+=len(dataset1[column])

dataset_new'''

dataset_new = dataset_new.sample(frac=1).reset_index(drop=True)
dataset_new

/home/guilherme/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,class1,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,...,native_county_ Scotland,native_county_ South,native_county_ Taiwan,native_county_ Thailand,native_county_ Trinadad&Tobago,native_county_ United-States,native_county_ Vietnam,native_county_ Yugoslavia,class2,classf
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,True,False,False,True,1
1,False,False,False,False,False,False,True,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,True,False,False,False,0
3,True,False,False,False,False,False,True,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,True,False,False,False,0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,True,False,False,False,0
6,True,False,False,False,True,False,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,True,False,False,False,0
8,False,False,False,True,False,False,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,True,False,False,False,0


In [12]:
'''indexes_null = (dataset_new['class1'].isnull())
dataset_new['class1'][indexes_null]=(np.random.randint(0,2,len(dataset_new['class1'][indexes_null]) ))

np.count_nonzero(dataset_new['class1'])/len(dataset_new['class1'])'''
dataset_new[['classf']]==0

,classf
0,False
1,False
2,True
3,False
4,True
5,True
6,False
7,True
8,False
9,True


In [13]:
dataset_new=dataset_new.fillna(0)
y = dataset_new[['classf']]
uniques = np.unique(dataset_new[['classf']])
for unique in uniques:
    name = str('classf')+'_'+str(unique)
    y[name]=(dataset_new[['classf']]==unique)
y = y.drop('classf',axis=1)
y

/home/guilherme/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,classf_0,classf_1,classf_2,classf_3
0,False,True,False,False
1,False,False,True,False
2,True,False,False,False
3,False,False,False,True
4,True,False,False,False
5,True,False,False,False
6,False,False,False,True
7,True,False,False,False
8,False,False,True,False
9,True,False,False,False


In [14]:
dataset_new = dataset_new.drop('class1',axis=1)
dataset_new = dataset_new.drop('class2',axis=1)
len(dataset_new)

23806

In [15]:
X = np.asarray(dataset_new)
y = np.asarray(y)
X_train, X_test, Y_train, Y_test   = train_test_split( X, y, test_size=0.33, random_state=42)




In [16]:
'''
Y0=to_categorical(Y0_train,num_classes=2)
Y1=to_categorical(Y1_train,num_classes=2)
Y0_test=to_categorical(Y0_test,num_classes=2)
Y1_test=to_categorical(Y1_test,num_classes=2)
'''

'\nY0=to_categorical(Y0_train,num_classes=2)\nY1=to_categorical(Y1_train,num_classes=2)\nY0_test=to_categorical(Y0_test,num_classes=2)\nY1_test=to_categorical(Y1_test,num_classes=2)\n'

In [32]:
a = Input(shape=(len(dataset_new.columns),))
x = Dense(8, activation='relu')(a) # this is your network, let's say you have 2 hidden layers of 64 nodes each (don't know if that's enough for you)
x = Dropout(0.5)(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dropout(0.5)(x)
#x = Dense(64, activation='relu')(x)
b1 = Dense(4,activation='softmax')(x)



model = Model(inputs=[a], outputs=[b1])
RMSprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
Adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00)
model.compile(loss=['binary_crossentropy'], optimizer=Adam, metrics=['accuracy'])
#model = Model(inputs=[a], outputs=[b1])
#model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])



In [24]:
filepath = './best_model.txt'
cb = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
model.fit(X_train, Y_train, epochs=15, batch_size=1, verbose=2, validation_data=(X_test,Y_test), callbacks=[cb])
#model.fit(X_train, [Y0,Y1], epochs=10, batch_size=1, verbose=2)

#model.fit(X, [Y0], epochs=100, batch_size=1, verbose=2)

Train on 15950 samples, validate on 7856 samples
Epoch 1/15
14s - loss: 2.6475 - acc: 0.8314 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 2/15
13s - loss: 2.6383 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 3/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 4/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 5/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 6/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 7/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 8/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 9/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 10/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 11/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2.6435 - val_acc: 0.8351
Epoch 12/15
13s - loss: 2.6382 - acc: 0.8354 - val_loss: 2

In [34]:
model.evaluate(X_test,Y_test)


6432/7856 [=======================>......] - ETA: 0s

[0.26410648882267673, 0.8326756619144603]

In [35]:

#np.count_nonzero(np.round(predictions[0][:,0]))/len(predictions[0][:,0]),np.count_nonzero(np.round(Y0_test[:,0]))/len(Y0_test[:,0])

In [36]:
dict_right={}
dict_right[0]=[]
dict_right[1]=[]
vec0=[]
predictions  = model.predict(X_test)
c0=c1=c2=c3=0
c0p=c1p=c2p=c3p=0
'''for row in range(len(predictions[0])):
    if np.argmax(predictions[0][row])==np.argmax(Y0_test[row]):
        dict_right[np.argmax(Y0_test[row])].append(1)
dict_right    
'''
for i in range(len(predictions)):
    
    A = np.asarray(predictions[i])
    maximum_indices = np.where(A==max(A))
    Y_pred = maximum_indices[0][0]
    Y_real = np.argmax(Y_test[i])
    #print (Y_pred,Y_real)
    if Y_real== 0:
        c0+=1
        if Y_real == Y_pred:
            c0p+=1
    elif Y_real==1:
        c1+=1
        if Y_real == Y_pred:
             c1p+=1          
    
    elif Y_real==2:
        c2+=1
        if Y_real == Y_pred:
            c2p+=1
        
    elif Y_real==3:
        c3+=1
        if Y_real == Y_pred:
            c3p+=1
print(c0p/c0)
print(c1p/c1)
print(c2p/c2)
print(c3p/c3)

0.5881898880741027
0.5021645021645021
1.0
0.8228527607361963


In [33]:
model.load_weights('./modelo0_5')

In [ ]:
dict_right={}
dict_right[0]=[]
dict_right[1]=[]
predictions  = model.predict(X_test)
for row in range(len(predictions[1])):
    if np.argmax(predictions[1][row])==np.argmax(Y1_test[row]):
        dict_right[np.argmax(Y1_test[row])].append(1)
dict_right   

In [ ]:
np.count_nonzero(np.round(predictions[1][:,0]))/len(predictions[1][:,0])
    

In [ ]:
cred = 

In [ ]:
#np.count_nonzero(np.round(cred['class']=='p'))/len(cred)
(Y_test[i])